**CARGAR LOS DATOS**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("attrition_availabledata_07.csv.gz")

X = df.drop(columns=["Attrition"])
y = df["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1/3, stratify=y, random_state=1725
)



## Evaluación inicial de modelos avanzados

Probamos los siguientes modelos con sus parámetros por defecto:

- Regresión logística (sin regularización y con L1)
- SVM lineal
- SVM con kernel RBF

Todos se evalúan con validación cruzada usando `balanced accuracy`, y se mide el tiempo que tarda en entrenarse cada uno.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, balanced_accuracy_score
import time

# Métrica y validación
scorer = make_scorer(balanced_accuracy_score)
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1725)

# Columnas
columnas_num = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
columnas_cat = X_train.select_dtypes(include="object").columns.tolist()

# Preprocesado
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), columnas_num),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), columnas_cat)
])

# Modelos (quitamos LogReg sin regularizar para evitar errores)
modelos = {
    "LogReg_L1": LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000),
    "SVM_lineal": LinearSVC(dual='auto', max_iter=10000),
    "SVM_RBF": SVC(kernel='rbf')
}

# Evaluación
for nombre, modelo in modelos.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("modelo", modelo)
    ])
    inicio = time.time()
    score = cross_val_score(pipe, X_train, y_train, cv=cv_inner, scoring=scorer).mean()
    fin = time.time()
    print(f"{nombre} → Balanced Accuracy: {score:.4f} | Tiempo: {fin - inicio:.2f} s")


LogReg_L1 → Balanced Accuracy: 0.6255 | Tiempo: 0.24 s
SVM_lineal → Balanced Accuracy: 0.5863 | Tiempo: 0.18 s
SVM_RBF → Balanced Accuracy: 0.6268 | Tiempo: 0.59 s


## Ajuste de hiperparámetros

Ajustamos los parámetros principales de cada modelo con `GridSearchCV`:

- `C` en Regresión Logística y SVM lineal
- `C` y `gamma` en SVM con kernel RBF


In [ ]:
from sklearn.model_selection import GridSearchCV

# LogReg L1
pipe_log_l1 = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000))
])
param_log_l1 = {"modelo__C": [0.01, 0.1, 1, 10, 100]}

grid_log_l1 = GridSearchCV(pipe_log_l1, param_log_l1, cv=cv_inner, scoring=scorer)
start = time.time()
grid_log_l1.fit(X_train, y_train)
end = time.time()
print("LogReg L1 - Mejor C:", grid_log_l1.best_params_, "| Balanced Accuracy:", grid_log_l1.best_score_, "| Tiempo:", round(end-start, 2), "s")

# SVM lineal
pipe_svm_lin = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", LinearSVC(dual="auto", max_iter=10000))
])
param_svm_lin = {"modelo__C": [0.01, 0.1, 1, 10, 100]}

grid_svm_lin = GridSearchCV(pipe_svm_lin, param_svm_lin, cv=cv_inner, scoring=scorer)
start = time.time()
grid_svm_lin.fit(X_train, y_train)
end = time.time()
print("SVM lineal - Mejor C:", grid_svm_lin.best_params_, "| Balanced Accuracy:", grid_svm_lin.best_score_, "| Tiempo:", round(end-start, 2), "s")

# SVM RBF
pipe_svm_rbf = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", SVC(kernel="rbf"))
])
param_svm_rbf = {
    "modelo__C": [0.1, 1, 10],
    "modelo__gamma": [0.001, 0.01, 0.1]
}

grid_svm_rbf = GridSearchCV(pipe_svm_rbf, param_svm_rbf, cv=cv_inner, scoring=scorer)
start = time.time()
grid_svm_rbf.fit(X_train, y_train)
end = time.time()
print("SVM RBF - Mejores params:", grid_svm_rbf.best_params_, "| Balanced Accuracy:", grid_svm_rbf.best_score_, "| Tiempo:", round(end-start, 2), "s")


LogReg L1 - Mejor C: {'modelo__C': 1} | Balanced Accuracy: 0.6255201511931533 | Tiempo: 2.64 s
SVM lineal - Mejor C: {'modelo__C': 1} | Balanced Accuracy: 0.5863068724722202 | Tiempo: 0.93 s
SVM RBF - Mejores params: {'modelo__C': 10, 'modelo__gamma': 0.1} | Balanced Accuracy: 0.8065023661254849 | Tiempo: 7.23 s


## Relevancia de atributos

La regresión logística permite consultar qué variables tienen más peso en la predicción.  
Mostramos los 10 atributos más importantes en términos de coeficientes (positivos o negativos).


In [ ]:
import numpy as np

modelo_final = grid_log_l1.best_estimator_
modelo_final.fit(X_train, y_train)

# Nombres tras one-hot
ohe = modelo_final.named_steps["preprocessor"].named_transformers_["cat"].named_steps["onehot"]
cat_feature_names = ohe.get_feature_names_out(columnas_cat)
final_feature_names = columnas_num + list(cat_feature_names)

# Coeficientes
coef = modelo_final.named_steps["modelo"].coef_.flatten()
importancia = pd.DataFrame({"Atributo": final_feature_names, "Peso": coef})
importancia["abs"] = importancia["Peso"].abs()
importancia.sort_values("abs", ascending=False).drop(columns="abs").head(10)


,Atributo,Peso
23,BusinessTravel_Non-Travel,-1.148882
47,MaritalStatus_Married,-1.046596
46,MaritalStatus_Divorced,-1.016199
22,YearsWithCurrManager,-0.764590
18,TotalWorkingYears,-0.675284
0,hrs,0.649956
41,JobRole_Manufacturing Director,-0.616789
28,Department_Sales,-0.608229
40,JobRole_Manager,-0.560847
27,Department_Research & Development,-0.489202


## Conclusiones – Modelos Avanzados

- La regresión logística con regularización L1 ha mostrado un rendimiento competitivo y permite interpretar qué atributos tienen mayor peso en la predicción.
- La SVM lineal ha dado resultados similares a la regresión logística, aunque su entrenamiento ha sido un poco más costoso computacionalmente.
- La SVM con kernel RBF ha sido la que mejor ha rendido tras el ajuste de hiperparámetros, aunque también ha sido la más lenta en entrenar.
- El ajuste de hiperparámetros ha mejorado claramente el rendimiento respecto a los valores por defecto.
- Los atributos más relevantes según los coeficientes del modelo logístico están relacionados principalmente con el puesto, el salario y la satisfacción del empleado.
